# Just a notebook for playing around with stuff

## calendar-analytics

In [1]:
from shared.googlelib import (
    authenticate,
    fetch_calendars,
    fetch_calendar_events
)
import pandas as pd

In [2]:
creds = authenticate(
    "client_secret_153639038451-8r2mq88ll6utdkb5fe2aacelccpl10mp.apps.googleusercontent.com.json"
)

In [21]:
calendars = fetch_calendars(creds)
events = fetch_calendar_events(creds, map(lambda x: x['summary'], calendars))

In [22]:
events_df = pd.json_normalize(events)

In [28]:
# Curate the events dataframe

# Convert the start.dateTime and end.dateTime column to a datetime object and localize it to Spain timezone
events_df["start.dateTime"] = pd.to_datetime(events_df["start.dateTime"], utc=True).dt.tz_convert("Europe/Madrid")
events_df["end.dateTime"] = pd.to_datetime(events_df["end.dateTime"], utc=True).dt.tz_convert("Europe/Madrid")

# Create duration columns
events_df["duration"] = pd.to_timedelta(events_df["end.dateTime"] - events_df["start.dateTime"])
events_df["duration_in_minutes"] = events_df["duration"].dt.total_seconds() / 60

# filter out events older than 60 days
events_60_df = events_df[events_df["start.dateTime"] > pd.Timestamp.now(tz="Europe/Madrid") - pd.DateOffset(days=60)]